## Amazon Sales Data Web Scrapping using Selenium

In [1]:
from selenium import webdriver
from lxml import etree
from time import sleep
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import random

options = webdriver.ChromeOptions()
options.add_argument('--headless')

HEADERS = ({'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.35',
            'Accept-Language': 'en-US, en;q=0.5','Accept-Encoding' : 'gzip', 'DNT' : '1','Connection' : 'close'})

In [ ]:
# url_list = []
# for page in range(51):
#     driver = webdriver.Chrome('chromedriver.exe', options=options)
#     driver.get('https://www.amazon.com/s?k=children%27s+book&i=stripbooks&rh=n%3A4%2Cp_n_feature_five_browse-bin%3A2578998011%7C2578999011%7C2579000011%7C2579001011&dc&page='+str(page)+'&crid=2V542SOL3EK2N&qid=1668618066&rnid=2245026011&sprefix=children%27s+book%2Caps%2C313&ref=sr_pg_4')
#     title_a_s = driver.find_elements_by_css_selector('[class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
#     for i in title_a_s:
#         url_list.append(i.get_attribute('href'))
#     driver.quit()

In [2]:
def get_publish_date(isbn):
    url = 'https://www.isbnsearcher.com/books/' + isbn.replace("-", "")
    webpage = requests.get(url, headers=HEADERS)
    soup = bs(webpage.content, "html.parser")
    for dt in soup.find_all('dt'):
        if "Published" in dt.text:
            published_date = dt.findNext('dd').text
            if ("-" in published_date):
                published_date = published_date.split('-')[0]
            return published_date
    return None

In [ ]:
# len(url_list)

In [ ]:
# df = pd.DataFrame(url_list)
# df.to_csv('URLs', index=False)

In [3]:
a = pd.read_csv('URLs.csv')
url_list = []
for i in a.values.tolist():
    url_list.append(i[0])

In [4]:
dataframe = pd.DataFrame(columns=['Title', 'Price', 'n_pages', 'n_customer_review', 'ISBN', 'publication_date'])

In [5]:
for i in url_list:
    try:
        i = i.replace("/zh/", "/en/")
        driver = webdriver.Chrome('chromedriver.exe', options=options)
        driver.get(i)
        title = driver.find_element_by_css_selector('[class="a-size-extra-large"]').text
        price = driver.find_element_by_css_selector('[class="a-size-base a-color-price a-color-price"]').text
        price = price.replace('US$', '')
        n_pages = driver.find_element_by_css_selector('#rpi-attribute-book_details-fiona_pages > div.a-section.a-spacing-none.a-text-center.rpi-attribute-value').text
        n_customer_review = driver.find_element_by_css_selector('#acrCustomerReviewText').text
        
        # get ISBN
        # auxiliary variables
        isbn_div_1 = driver.find_element_by_css_selector('[id = "detailBulletsWrapper_feature_div"]')
        isbn_div_2 = isbn_div_1.find_element_by_css_selector('[id = "detailBullets_feature_div"]')
        isbn_li_list = isbn_div_2.find_elements_by_css_selector('[class="a-list-item"]')
        isbn_content = isbn_li_list[4].text.strip()
        if isbn_content[:7] == 'ISBN-13':
            isbn = isbn_content
        else:
            for li in isbn_li_list:
                if li.text[:7].strip() == 'ISBN-13':
                    isbn = li.text
                    break
        isbn = isbn.replace('ISBN-13 : ', '')
                    
        sleep(random.uniform(1,2))
        publication_date = get_publish_date(isbn.strip()[-14:])
        if publication_date == None:
            continue
        
        df_dict = {'Title':[title], 'Price':[price], 'n_pages': [n_pages], 'n_customer_review':[n_customer_review],
                   'ISBN':[isbn], 'publication_date': [publication_date]}
        df = pd.DataFrame(df_dict)

        dataframe = pd.concat([dataframe, df], ignore_index=True)
        sleep(random.uniform(1,2))
        driver.quit()

    except:
        continue

In [6]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Title              371 non-null    object
 1   Price              371 non-null    object
 2   n_pages            371 non-null    object
 3   n_customer_review  371 non-null    object
 4   ISBN               371 non-null    object
 5   publication_date   371 non-null    object
dtypes: object(6)
memory usage: 17.5+ KB


In [7]:
dataframe.head()

,Title,Price,n_pages,n_customer_review,ISBN,publication_date
0,The Wonderful Things You Will Be,$8.55,36 pages,"25,800 ratings",978-0385376716,2015
1,"World of Eric Carle, Around the Farm 30-Button...",$11.57,24 pages,"18,566 ratings",978-1450805759,2011
2,Where the Wild Things Are,$13.20,48 pages,"32,473 ratings",978-1450805759,2011
3,Kindness is my Superpower: A children's Book A...,$11.97,38 pages,"4,574 ratings",979-8669081621,2020
4,The Rainbow Fish,$12.99,32 pages,"19,262 ratings",978-1558580091,1992


In [8]:
dataframe.to_csv('amazon_newly_updated.csv', index=False)

In [9]:
len(dataframe)

371

## Data Analysis

In [ ]:
# import pandas as pd

In [ ]:
# sales_data = pd.read_csv('amazon_newly_updated.csv')
# reviews = pd.read_csv('review.csv', names=["ISBN","review"])

In [ ]:
# df = pd.merge(sales_data, reviews, on="ISBN", how ="left")

In [ ]:
# print("shape of dataset before merge", sales_data.shape)
# print("shape of dataset frame after merge,", df.shape)